# Valley Bottom Extraction
## Iterate over regional outputs and select input datasets for RVBD tool as well as convert/create necessary datasets.

Create data dictionary to store input parameters for tool.
1. River Network
2. Catchments
3. DEM
4. Wetlands - as feature class or shapefile
5. Output Folder - Directory
6. River Valley Bottom - Output River Valley Bottom Polygon

In [1]:
# Import modules
import arcpy
import os
import sys
# Import RVBD modules
import os
from pathlib import Path
import datetime
from data_preparation.RVBD.slope import slope
from data_preparation.RVBD.valleycostdistance import valley
from data_preparation.RVBD.costdistance import costdist

print('imports complete')
print(f'sys paths {sys.path}')

imports complete
sys paths ['C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\data_preparation\\sensitivity_drivers\\geomorphology', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python37.zip', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\Resources\\ArcToolbox\\Scripts', 'C:\\Users\\dwmerrigan\\AppDat

In [2]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value

In [3]:
arcpy.env.overwriteOutput = True
# Set data dir equal to directory containing the AKSSF regional sub-folders.
data_dir = r"D:\\GIS\\AKSSF"
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
# Regions to be processed
rList = ['Kodiak','Prince_William_Sound','Copper_River','Bristol_Bay','Cook_Inlet']
regions = arcpy.ListWorkspaces(workspace_type="Folder")
regions = [os.path.normpath(r) for r in regions if os.path.basename(r) in rList]
#working gdb to save intermediate outputs
rvbd_scratchgdb = r"U:\\RVBD_Outputs\\rvbd_scratch.gdb"
rvbd_scratchfol = r"U:\\RVBD_Outputs"
print (regions)

#Create rvbd scratch folders if they do not already exist
import os, arcpy

rvbd_scratchfol = r"U:\\RVBD_Outputs"
if not arcpy.Exists(rvbd_scratchfol):
    os.mkdir(rvbd_scratchfol)
    print(f'Creating scratch folder {rvbd_scratchfol}')
else:
    print(f'Scratch folder {rvbd_scratchfol} already exists')
print('----------')

if not arcpy.Exists(rvbd_scratchgdb):
    arcpy.CreateFileGDB_management(rvbd_scratchfol,'rvbd_scratch.gdb')
    print(f'Creating scratch gdb {rvbd_scratchgdb}')
else:
    print(f'Scratch folder {rvbd_scratchgdb} already exists')
print('----------')

#Create Regional output folders
for region in regions:
    rname = os.path.basename(region)
    rvbdscratchpath = os.path.join(rvbd_scratchfol, rname + '_RVBD')
    if not arcpy.Exists(rvbdscratchpath):
        os.mkdir(rvbdscratchpath)
        print (f'{rname} does not have a scratch folder')
        print (f'Creating river valley bottom scratch folder at  {rvbdscratchpath}')
    else:
        print(f'Scratch folder for {rname} already created at {rvbdscratchpath}')

    print('----------')
print(f'All scratch workspaces set')


['D:\\GIS\\AKSSF\\Bristol_Bay', 'D:\\GIS\\AKSSF\\Cook_Inlet', 'D:\\GIS\\AKSSF\\Copper_River', 'D:\\GIS\\AKSSF\\Kodiak', 'D:\\GIS\\AKSSF\\Prince_William_Sound']
Scratch folder U:\\RVBD_Outputs already exists
----------
Scratch folder U:\\RVBD_Outputs\\rvbd_scratch.gdb already exists
----------
Scratch folder for Bristol_Bay already created at U:\\RVBD_Outputs\Bristol_Bay_RVBD
----------
Scratch folder for Cook_Inlet already created at U:\\RVBD_Outputs\Cook_Inlet_RVBD
----------
Scratch folder for Copper_River already created at U:\\RVBD_Outputs\Copper_River_RVBD
----------
Scratch folder for Kodiak already created at U:\\RVBD_Outputs\Kodiak_RVBD
----------
Scratch folder for Prince_William_Sound already created at U:\\RVBD_Outputs\Prince_William_Sound_RVBD
----------
All scratch workspaces set


### Create data dictionary to use as input to RVBD script.
 * Collect and convert datasets as necessary to use as inputs for the RVBD script
 * Create region specific output folders if they do not already exist.


In [4]:
rvbdDict = {}

# Walk through gdbs and set input data sets. Convert wetland raster to shapefile if this has not already been done.
akssf_gdb = r'D:\GIS\AKSSF_land_met\AKSSF_land_met.gdb'
for region in regions:
    rname = os.path.basename(region)
    print(rname)
    arcpy.env.workspace = region
    gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
    outgdb = gdb[0]
    walk = arcpy.da.Walk(region, datatype=['RasterDataset','FeatureClass'])
    rvbdscratchpath = os.path.join(rvbd_scratchfol, rname + '_RVBD')
    for dirpath, dirnames, filenames in walk:
        for filename in filenames:
            if 'streams_merge' == filename or 'NHDFlowline_merge' == filename:
                strsource = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,strsource)
            # Set merged watersheds dataset
            elif 'wtds_merge' == filename:
                wtdsource = os.path.join(dirpath, filename)
                # Dissolve Watersheds - keep as single part
                wtd_dis = os.path.join(outgdb,rname + '_wtd_dis')
                if not arcpy.Exists(os.path.join(outgdb,rname + '_wtd_dis')):
                    arcpy.Dissolve_management(wtdsource,wtd_dis,None, None, "SINGLE_PART", "DISSOLVE_LINES")
                    # Append dissolved watersheds to Dictionary to use as input for rvbd tool
                append_value(rvbdDict,rname,wtd_dis)
            # Set elev raster
            elif 'elev.tif' == filename:
                elev_path = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,elev_path)
            # Set slope raster
            elif 'slope.tif' == filename:
                slope_path = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,slope_path)
            # Set aspect raster
            elif 'aspect.tif' == filename:
                asp_path = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,asp_path)

            # # Set wetland raster
            # elif 'wetlands.tif' == filename:
            #     wras_path = os.path.join(dirpath, filename)
            #     wrasname = filename[:-4]
            #     wetfcpath = os.path.join(outgdb, wrasname)
            #     wetfcpath2 = os.path.join(rvbd_scratchgdb, wrasname)
            #     # Make local copy projected in AKAlbers
            #     if not arcpy.Exists(wetfcpath):
            #         print(f'Converting {wrasname} to feature class...')
            #         wetfc_conv = arcpy.RasterToPolygon_conversion(wras_path,wetfcpath2,raster_field='VALUE',
            #                                                  simplify='NO_SIMPLIFY',
            #                                                  create_multipart_features='MULTIPLE_OUTER_PART')
            #         wetfc = arcpy.FeatureClassToFeatureClass_conversion(wetfcpath2,outgdb,wrasname,
            #                                                             where_clause='gridcode = 1')
            #     else:
            #         print(f'{wetfcpath} already created')
            #         wetfc = wetfcpath
            else:
                continue

    # Set NLCD and try with different wetland categories
    nlcd_wet = r"D:\\Basedata\\NLCD_2016_Land_Cover_AK_20200724\\NLCD_2016_Land_Cover_AK_20200724.img"
    wetname = 'rvbd_wetlands'
    reclassname = '_nlcd_wet_reclass_v2'
    wetfcpath = os.path.join(outgdb, wetname)
    wet_reclass_path = os.path.join(outgdb, reclassname)

    # Make local copy projected in AKAlbers
    if not arcpy.Exists(wetfcpath):
        from arcpy.sa import *
        # Start time
        start = datetime.datetime.now()
        print("Wetland Classification for RVCBD started at %s" % start)
        arcpy.AddMessage ("Wetland Classification for RVCBD started at %s" % start)
        arcpy.AddMessage ("\n")

        arcpy.env.mask = wtd_dis
        wet_remap =  "'Unclassified' 0;'Open Water' 1;'Perennial Ice/Snow' 0;'Developed, Open Space' 0;'Developed, Low Intensity' 0;'Developed, Medium Intensity' 0;'Developed, High Intensity' 0;'Barren Land' 0;'Deciduous Forest' 0;'Evergreen Forest' 0;'Mixed Forest' 0;'Dwarf Shrub' 0;Shrub/Scrub 0;Grassland/Herbaceous 0;Sedge/Herbaceous 1;Moss 0;Pasture/Hay 0;'Cultivated Crops' 0;'Woody Wetlands' 1;'Emergent Herbaceous Wetlands' 1"
        wet_reclass = arcpy.Reclassify_3d(nlcd_wet, "NLCD Land Cover Class", wet_remap,wet_reclass_path, "NODATA")
        print(f'Converting {nlcd_wet} to feature class...')
        wetfc_conv = arcpy.RasterToPolygon_conversion(wet_reclass,os.path.join(r'in_memory','wetname'),raster_field='VALUE',
                                                 simplify='NO_SIMPLIFY',
                                                 create_multipart_features='MULTIPLE_OUTER_PART')
        wetfc = arcpy.FeatureClassToFeatureClass_conversion(wetfc_conv,outgdb,wetname,
                                                            where_clause='gridcode = 1')
        # End time
        end = datetime.datetime.now()
        print("Wetland Classification for RVCBD finished at %s" % end)
        arcpy.AddMessage ("Wetland Classification for RVCBD finished at %s" % end)
        arcpy.AddMessage ("\n")
        time_elapsed = end - start
        print("Time elapsed %s" % (time_elapsed))
        arcpy.AddMessage ("Time elapsed %s" % time_elapsed)
        arcpy.AddMessage ("\n")
        print(".........................................................")
        arcpy.AddMessage (".........................................................")
        arcpy.AddMessage ("\n")
    else:
        print(f'{wetfcpath} already created')
        wetfc = wetfcpath

    append_value(rvbdDict,rname,wetfcpath)
    append_value(rvbdDict,rname,rvbdscratchpath)
    rivbottpath = os.path.join(akssf_gdb,rname + '_riv_valbot_V3')
    append_value(rvbdDict,rname,rivbottpath)
    print(f'Stream input = {strsource}')
    print(f'Watersheds input = {wtd_dis}')
    print(f'DEM = {elev_path}')
    print(f'Slope input = {slope_path}')
    print(f'Aspect input = {asp_path}')
    print(f'Wetlands input = {wetfc}')
    print('----------')
print('----------')
print('Data Dictionary created')


Bristol_Bay
D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\rvbd_wetlands already created
Stream input = D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\streams_merge
Watersheds input = D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\Bristol_Bay_wtd_dis
DEM = D:\GIS\AKSSF\Bristol_Bay\elev.tif
Slope input = D:\GIS\AKSSF\Bristol_Bay\slope.tif
Aspect input = D:\GIS\AKSSF\Bristol_Bay\aspect.tif
Wetlands input = D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\rvbd_wetlands
----------
Cook_Inlet
D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\rvbd_wetlands already created
Stream input = D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\NHDFlowline_merge
Watersheds input = D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\Cook_Inlet_wtd_dis
DEM = D:\GIS\AKSSF\Cook_Inlet\elev.tif
Slope input = D:\GIS\AKSSF\Cook_Inlet\slope.tif
Aspect input = D:\GIS\AKSSF\Cook_Inlet\aspect.tif
Wetlands input = D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\rvbd_wetlands
----------
Copper_River
D:\GIS\AKSSF\Copper_River\Copper_River.gdb\rvbd_wetlands already created
Stream inpu

### Begin RVBD
RVBD script adapted from code developed for <a href="https://www.mdpi.com/2073-4441/13/6/827/htm">A Stepwise GIS Approach for the Delineation of River Valley Bottom within Drainage Basins Using a Cost Distance Accumulation Analysis</a>

Consider using the <a href = "https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/distance-accumulation.htm">Distance Accumulation Tool</a> instead of Cost Distance as this tool is recommend by ESRI over Cost Distance (Similar to using Surface Parameters instead of Slope)

NLCD - Include additional wetland classes
    * v1 uses wetlands and open water
    * v2 uses wetlands, open water, and sedge herbaceous
    * uses wetlands, open water, and sedge herbaceous  with new slope 3x3

Valley bottoms are extracted from cost distance accumulation raster as follows:
1. Filter cost distance raster to values >0 and <=500
2. Calculate mean cost distance from wetlands identified by creating theissan polygons from points spaced 100 meters along the river network.
3. Select all cells <= threshold value and export as valley bottom raster

Script has no process by which River Valleys can be identified for watersheds that do not have calibration wetlands.
    * Threshold value ???

There are some areas where lakes are left as "holes" in the river valley polygon where they should be part of the valley bottom.
 * ~~Suggestion - Select lakes that intersect the stream network and merge with River Valley Bottom polygons.~~
 * Original Slope raster produced using surface parameters with 5x5 moving window - Rerun with 3x3 or less and use as input for slope.

Something wrong with cook inlet.  The output river valley bottom is just the footprint of the slope raster for the entire region.  Possibly something wrong with No-Data cells? Need to investigate further.
 * 0 values in slope raster causing issue

Valley bottom length
Confined or unconfined


In [5]:
rvbdDict


{'Bristol_Bay': ['D:\\GIS\\AKSSF\\Bristol_Bay\\aspect.tif',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\elev.tif',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\slope.tif',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\streams_merge',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\Bristol_Bay_wtd_dis',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\rvbd_wetlands',
  'U:\\\\RVBD_Outputs\\Bristol_Bay_RVBD',
  'D:\\GIS\\AKSSF_land_met\\AKSSF_land_met.gdb\\Bristol_Bay_riv_valbot_V3'],
 'Cook_Inlet': ['D:\\GIS\\AKSSF\\Cook_Inlet\\aspect.tif',
  'D:\\GIS\\AKSSF\\Cook_Inlet\\elev.tif',
  'D:\\GIS\\AKSSF\\Cook_Inlet\\slope.tif',
  'D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\NHDFlowline_merge',
  'D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\Cook_Inlet_wtd_dis',
  'D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\rvbd_wetlands',
  'U:\\\\RVBD_Outputs\\Cook_Inlet_RVBD',
  'D:\\GIS\\AKSSF_land_met\\AKSSF_land_met.gdb\\Cook_Inlet_riv_valbot_V3'],
 'Copper_River': ['D:\\GIS\\AKSSF\\Copper_River\\aspect.tif',
  'D:\\GIS\\A

In [8]:

# Starting script
start = datetime.datetime.now()
print("Script started at %s" % start)
arcpy.AddMessage ("Script started started at %s" % start)
arcpy.AddMessage ("\n")

# Check out spatial extention
arcpy.CheckOutExtension("Spatial")

# Environments
arcpy.env.overwriteOutput = True

# #try with test data
# # Python inputs
# riv =  r"D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\streams_merge" # river network as a shapefile
# cat = r"D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\wtds_merge" # catchments as a shapefile
# dem = r"D:\\GIS\\AKSSF\\Kodiak\\elev.tif"# elevation model as a raster
# wet = r"D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\wetlands" # measured valley bottom, wetlands or any wet signature in low areas
# pth = r"D:\\GIS\\RVBD_scratch\\Kodiak_RVBD" # path to outputs (scratch and cost distance folders)
# outval = r"D:\\GIS\\RVBD_scratch\\Kodiak_RVBD\\Kodiak_RVBD.gdb\\kod_rvbd" # path to output river valley
# slp_in = r"D:\\GIS\\AKSSF\\Kodiak\\slope.tif"

regions = ['D:\\GIS\\AKSSF\\Copper_River',
           'D:\\GIS\\AKSSF\\Prince_William_Sound',
           'D:\\GIS\\AKSSF\\Bristol_Bay',
           'D:\\GIS\\AKSSF\\Cook_Inlet']
#            #regions = ['D:\\GIS\\AKSSF\\Kodiak']
# need to iterate over regions and inputs from data dictionary

for region in regions:
    key = os.path.basename(region)
    print(key)
    pth = rvbdDict[key][6]
    # Check to see if scratch geodatabase exists
    s = Path(os.path.join(pth, "outputs.gdb"))
    if s.exists() == False:
        scrg = arcpy.CreateFileGDB_management(pth, "outputs")
        scr = scrg.getOutput(0) # scratch folder
        print(f'Scratch gdb for {key} set to {scr}...')
    else:
        scr = os.path.join(pth, "outputs.gdb") #scr = r"in_memory" (if you want to store in memory)
        print(f'Creating scratch gdb for {key} at {scr}...')

    # Check to see if cost dist folder exists (use code in comments if you want results in a folder instead of gdb)
    c = Path(os.path.join(pth, "costdist.gdb")) # Path(os.path.join(pth, "costdist"))
    if c.exists() == False:
        cstg = arcpy.CreateFileGDB_management(pth, "costdist")  # arcpy.CreateFolder_management(pth, "costdist")
        cst = cstg.getOutput(0) # cost distance folder
        print(f'Cost gdb for {key} set to {cst}...')
    else:
        cst = os.path.join(pth, "costdist.gdb") # os.path.join(pth, "costdist")
        print(f'Creating cost gdb for {key} at {cst}...')

    print('----------')
    dem = rvbdDict[key][1]
    # # Use modified slope script to calculate new slope raster with surface param
    # slp_in = rvbdDict[key][2]
    slp_in = os.path.join(scr, "slope") # Path to output from slope script using 3x3 neighborhood
    asp_in = rvbdDict[key][0]
    riv = rvbdDict[key][3]
    cat = rvbdDict[key][4]
    wet = rvbdDict[key][5]
    outval = rvbdDict[key][7]

    print(f'riv = {riv}')
    print(f'cat = {cat}')
    print(f'dem = {dem}')
    print(f'wet = {wet}')
    print(f'pth = {pth}')
    print(f'outval = {outval}')
    print(f'slp_in = {slp_in}')
    print(f'asp_in = {asp_in}')
    print('----------')

    # Create new slope raster using smaller default neighborhood distance of 3x3
    if not arcpy.Exists(slp_in):
        slope(riv, cat, dem, scr)
        print(f'Slope raster not calcuated - Running slope script...')
    else:
       print(f'Using slope raster {slp_in} for cost distance analysis')

    # Cost distance calculation - Began setting up for distance accumulation in case we decide to pursue this further
    costdist(riv, cat, slp_in, cst, scr, asp_in, dem)
    print (f'Cost distance raster not calculated - Running Cost distance accumulation script...')

    # Extract valley
    if not arcpy.Exists(outval):
        valley(riv, wet, cst, scr, outval)
        print(f'Valley Bottom Polygon not yet calculated - Running Valley cost distance script...')
    else:
        continue

# Ending script
end = datetime.datetime.now()
print("Script ended at %s" % end)
arcpy.AddMessage ("Script ended at %s" % end)
arcpy.AddMessage ("\n")
time_elapsed = end - start
print("Time elapsed %s" % (time_elapsed))
arcpy.AddMessage ("Time elapsed %s" % time_elapsed)
arcpy.AddMessage ("\n")
print(".........................................................")
arcpy.AddMessage (".........................................................")
arcpy.AddMessage ("\n")

Script started at 2022-01-13 18:34:34.167506
Copper_River
Creating scratch gdb for Copper_River at U:\\RVBD_Outputs\Copper_River_RVBD\outputs.gdb...
Creating cost gdb for Copper_River at U:\\RVBD_Outputs\Copper_River_RVBD\costdist.gdb...
----------
riv = D:\GIS\AKSSF\Copper_River\Copper_River.gdb\NHDFlowline_merge
cat = D:\GIS\AKSSF\Copper_River\Copper_River.gdb\Copper_River_wtd_dis
dem = D:\GIS\AKSSF\Copper_River\elev.tif
wet = D:\GIS\AKSSF\Copper_River\Copper_River.gdb\rvbd_wetlands
pth = U:\\RVBD_Outputs\Copper_River_RVBD
outval = D:\GIS\AKSSF_land_met\AKSSF_land_met.gdb\Copper_River_riv_valbot_V3
slp_in = U:\\RVBD_Outputs\Copper_River_RVBD\outputs.gdb\slope
asp_in = D:\GIS\AKSSF\Copper_River\aspect.tif
----------
Slope calculation started at 2022-01-13 18:34:34.684754
Completed calculating slope using surface parameters tool at 2022-01-13 18:44:14.216840
Slope calculation ended at 2022-01-13 18:44:14.216840
Time elapsed 0:09:39.532086
...............................................